### 1 player convo

In [ ]:
import json
from rich.console import Console
from rich.table import Table

file_path = '../'
# file_path += 'DATA/RUN_20240731_120018/ITERATION_1/P0_GAME_1_2_20240731_120018_p0.json'
file_path += 'outputs/2024-08-15/17-45-16/iteration_01/p0_iter_01_game_0001.json'

table_name = 'player_example'
table_name += '.html'
to_html = True


with open(file_path, 'r') as f:
    conversation = json.load(f)

no_rules = True

# Initialize the console
console = Console(record=to_html)

# Create a table to display the conversation
table = Table(show_header=True, header_style="bold magenta")
table.add_column("Role", style="dim", width=12)
table.add_column("Content" )


for message in conversation:
    match message["role"]:
      case 'assistant':
        style = "blue"
      case _:
        style = "red"
    if message["role"] == 'user':
      table.add_row('Moderator', message["content"], style = style)
    else:
      table.add_row('Player', message["content"], style = style)
    table.add_row("", "")
table.add_section()

console.print(table)
if to_html:
  html = console.export_html()

  custom_css = """
  <style>
      body {
          font-size: 12px;
      }
  </style>
  """

  html = html.replace("<head>", f"<head>{custom_css}")

  with open(table_name, "w") as f:
      f.write(html)

### 2 players convo

In [1]:
from itertools import zip_longest
import json
import heapq
from rich.console import Console
from rich.table import Table

iter = '01'
game = '0001'
file_path = '../'
file_path += 'outputs/2024-08-27/15-23-44'

player0_path = file_path + f'/iteration_{iter}/p0_iter_{iter}_game_{game}.json'
player1_path = file_path + f'/iteration_{iter}/p1_iter_{iter}_game_{game}.json'

table_name = 'agreement_example'
table_name += '.html'
to_html = True


with open(player0_path, 'r') as f:
    convo_p0 = [j['content'] for j in json.load(f)]

with open(player1_path, 'r') as f:
    convo_p1 = [j['content'] for j in json.load(f)]

def is_moderator(item):
    return item.startswith('The other player') or item.startswith('A final proposal has been made')

def filter_elements(lst):
    lst = lst[1:]
    last_element = lst[-1]
    result = []
    for id, item in enumerate(lst):
        if is_moderator(item):
            result.append(lst[id-1])
    
    result.append(last_element)
    return result

def extract_content(tag, text):
    start_tag = f"<{tag}>"
    end_tag = f"</{tag}>"
    if start_tag not in text:
        return ""
    start_index = text.find(start_tag) + len(start_tag)
    end_index = text.find(end_tag)
    return text[start_index:end_index].strip()

# Filtered lists
filtered_l1 = filter_elements(convo_p0)
filtered_l2 = filter_elements(convo_p1)

# Combine the lists
convo = [item for pair in zip_longest(filtered_l1, filtered_l2) for item in pair if item is not None]
# convo = []
# for a, c in zip_longest(filtered_l1, filtered_l2):
#     convo.extend([a, c])

# convo = list(heapq.merge(filtered_l1, filtered_l2))

# Initialize the console
console = Console(record=to_html)

# Create a table to display the conversation
table = Table(show_header=True, header_style="bold magenta")
table.add_column("Role", style="dim", width=12)
table.add_column("Content" )


table.add_row('Moderator', convo_p0[0], style = "green")
table.add_row("", "")
for id, content in enumerate(convo):
    reasoning = extract_content("reason", content)
    message = extract_content("message", content) or extract_content("finalize", content)
    if id > 0:
        table.add_row('Moderator', "(prompt)", style = "dim green")
        table.add_row("", "")
    if id % 2 == 0:
      table.add_row('Player 0', reasoning, style = "dim blue")
      table.add_row("", "")
      table.add_row('', message, style = "blue")
    else:
      table.add_row('Player 1', reasoning, style = "dim red")
      table.add_row("", "")
      table.add_row('', message, style = "red")
    table.add_row("", "")
table.add_section()

console.print(table)

if to_html:
    html = console.export_html()

    custom_css = """
    <style>
        body {
            font-size: 12px;
        }
    </style>
    """

    html = html.replace("<head>", f"<head>{custom_css}")

    with open(table_name, "w") as f:
        f.write(html)


┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Role         ┃ Content                                                                                          ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Moderator    │ You are a player in a two-player negotiation game. I, the user, will be the game coordinator,    │
│              │ acting as a middleman between you and the other player. Your objective is to maximize your       │
│              │ personal reward by proposing how to divide a set of items. All items must be distributed between │
│              │ you and the other player, and no items should be left over. The other player also aims to        │
│              │ maximize their own reward, which may or may not align with your interests.                       │
│              │                                                                                                  │
│              │ The reward you are trying to maximize is calculated as follow: your utility values multiplied by │
│              │ items you take                                                                                   │
│              │                                                                                                  │
│              │ Game Mechanics:                                                                                  │
│              │ Turn-taking: You and the other player will take turns exchanging one message at a time. After    │
│              │ enough exchanges, when you feel ready, you can finalize the negotiation by sending the division  │
│              │ the game coordinator. Once a player decides to send a final division, the other player must also │
│              │ send a final division, ending the game.                                                          │
│              │ Reasoning and Action: At the start of your turn, you will be asked to reason about the current   │
│              │ state of the game and how to fulfill your objective, then make an action (either messaging your  │
│              │ the other player or finalize the negotiation).                                                   │
│              │ Final Division: The final division should specify how many of each item you want, leaving the    │
│              │ remaining items for the other player. It should be JSON parsable. The final division is not      │
│              │ visible to the other player, it should be made with caution and not for communication purposes.  │
│              │ Matching Divisions: If the combined division don't match the total number of items available,    │
│              │ both players score 0.                                                                            │
│              │                                                                                                  │
│              │ Formatting:                                                                                      │
│              │ Reasoning: <reason> [Your reasoning here.] </reason>                                             │
│              │ Messages: <message> [Your message here.] </message>                                              │
│              │ Final division: <finalize>{ "i_take": {"books":x, "hats":y, "balls":z}, "other_player_gets":     │
│              │ {"books":a, "hats":b, "balls":c} }</finalize>, where 'i_take' is your share and                  │
│              │ 'other_player_gets' is the other player share of the items.                                      │
│              │                                                                                                  │
│              │ Only do one reasoning + action per turn.                                                         │
│              │                                        